In [ ]:
import json
from ExchangeRateEvent import *
from get_currencies import *
from constants import API_URL, BASE_CURRENCY, LOCAL_CURRENCIES, SALESFORCE_INSTANCE_URL, SALESFORCE_API_USER, SALESFORCE_API_PASS, SALESFORCE_API_TOKEN, DB_URL
import logging
import os
import sys
from datetime import timedelta, date
import datetime
import pandas as pd
from file_opener import *
from simple_salesforce import Salesforce
from dateutil.parser import parse

In [ ]:
#Navigate and import from the submodule
sys.path.append('..\..')
from data_utils.sf_connect import sf_connect
from data_utils.file_opener import file_opener

In [ ]:
API_KEY = os.environ['API_KEY']

In [ ]:
file_date = datetime.datetime.now().strftime('%Y%m%d')

In [ ]:
donation_soql = file_opener('donations.soql')

In [ ]:
sf = sf_connect()

In [ ]:
sf.sf_query_object(donation_soql)

In [ ]:
renamer = {'Id': 'opportunity_id'
           ,'Currency__c': 'local_currency'
           ,'Local_Currency_Amount_Number_For_Nat__c': 'local_currency_amount'
           ,'CloseDate': 'exchange_rate_date'}

In [ ]:
sf_donations_df = sf.data

In [ ]:
if len(sf_donations_df) > 0:
    sf_donations_df.rename(columns = renamer, inplace = True)
    currencies_dates = sf_donations_df[['local_currency', 'exchange_rate_date']].copy().drop_duplicates()

In [ ]:
print('The Salesforce returned a dataframe with {} records.'.format(len(sf_donations_df)))

In [ ]:
if len(currencies_dates) > 0:
    records = []
    for c in currencies_dates.iterrows():
        data = c[1]
        local_currency = data[0]
        date = data[1].strftime('%Y%m%d')

        exchange_rate = get_currencies(
                                       BASE_CURRENCY
                                       ,local_currency
                                       ,date
                                       ,API_KEY
                                       ,API_URL
                                       )
        records.append(exchange_rate)

In [ ]:
data = [rows.__dict__ for days in records for rows in days]

In [ ]:
if len(data) > 0:
    all_exchange_rates = pd.DataFrame(data)
    all_exchange_rates['exchange_rate_date'] = all_exchange_rates.apply(lambda x: parse(x['exchange_rate_date']).date(), axis = 1)

In [ ]:
if len(all_exchange_rates) > 0 and len(sf_donations_df) > 0:
    conversion = sf_donations_df.merge(all_exchange_rates[['local_currency', 'exchange_rate_date', 'exchange_rate']]
                                       ,how = 'left'
                                       ,on = ['local_currency', 'exchange_rate_date']
                                       ,indicator = True)
    
    conversion['base_currency_amount'] = conversion.apply(lambda x: round(x['local_currency_amount']/x['exchange_rate'], 2), axis = 1)
    
    print('{} records were successfuly converted.'.format(len(conversion[conversion['_merge'] == 'both'])))
    
    conversion[conversion['_merge'] == 'both'].to_csv('currency_conversion_{}.csv'.format(file_date))
    
    conversion[conversion['_merge'] != 'both'].to_csv('no_currency_conversion_{}.csv'.format(file_date))